In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [7]:
import json
lion = json.load(open('lion.geojson'))

In [62]:
lion_df = pd.DataFrame(f['properties'] for f in lion['features'])

In [104]:
lion_l = lion_df.filter(['FromLeft', 'FromRight', 'ToLeft', 'ToRight', 'SegmentID', 'StreetCode'], axis=1).copy()
lion_r = lion_l.copy()
lion_l['From'] = lion_l['FromLeft']
lion_l['To'] = lion_l['ToLeft']
lion_l['Side'] = 'L'
lion_r['From'] = lion_r['FromRight']
lion_r['To'] = lion_r['ToRight']
lion_r['Side'] = 'R'

lion_split = lion_l.append(lion_r)
lion_split['Even'] = (lion_split['From'] % 2 == 0)
lion_split = lion_split.filter(['StreetCode', 'SegmentID', 'From', 'To', 'Side', 'Even'])

In [105]:
lion_split = lion_split.sort_values(['From', 'StreetCode', 'Even'])

In [10]:
parking_violations = pd.read_csv('parking-violations.csv', nrows=100000, low_memory=False)

In [116]:
CONVERT_BOROUGH = {
    'K': 3,
    'Q': 4,
    'M': 1,
    'B': 2,
    'R': 5,
}
def get_full_street_code(row):
    stc1 = int(row['Street Code1'])
    if stc1 == 0:
        return -1
    if row['Violation County'] not in CONVERT_BOROUGH:
        return -1
    return str(stc1 + CONVERT_BOROUGH[row['Violation County']] * 100000)

def addr(row):
    addr_str = str(row['House Number'])
    try:
        if addr_str.find('-') >= 0:
            above, below = addr_str.split('-')
            addr = int(above) * 1000 + int(below)
        elif addr_str == 'nan':
            return -1
        else:
            addr = int(addr_str)
    except ValueError:
        # TODO: try to clean up non-numeric addresses
        return -1
    return addr

parking_violations['StreetCode'] = parking_violations.apply(get_full_street_code, axis=1)
parking_violations['Addr'] = parking_violations.apply(addr, axis=1)
parking_violations['Even'] = (parking_violations['Addr'] % 2 == 0)

In [122]:
parking_violations_cleaned = parking_violations[
    (parking_violations['Addr'] > -1) &
    parking_violations['StreetCode'].notna()].copy().sort_values(
    ['Addr', 'StreetCode', 'Even'])

228            1
266            1
556            1
557            1
749            1
          ...   
97867    1943055
26180    1945049
22644    2331033
95869    2989091
8560     5720024
Name: Addr, Length: 39266, dtype: int64

In [125]:
pv_merged = pd.merge_asof(
    parking_violations_cleaned,
    lion_split, 
    by=['StreetCode', 'Even'], 
    right_on='From', left_on='Addr')
pv_merged = pv_merged[pv_merged['Addr'] <= pv_merged['To']]
pv_merged

In [110]:
import shapely.geometry
lion_geom = {
    f['properties']['SegmentID'] : shapely.geometry.MultiLineString(f['geometry']['coordinates'])
    for f in lion['features']
}

In [33]:
from pyproj import CRS, Transformer
wgs84 = CRS.from_epsg(4326)
nySP = CRS.from_epsg(2263)
transformer = Transformer.from_crs(nySP, wgs84)

In [126]:
def find_latlng(row):
    if row['To'] == row['From']:
        fraction = 0
    else:
        fraction = (row['Addr'] - row['From']) / (row['To'] - row['From'])
    coords = lion_geom[row['SegmentID']].interpolate(fraction, True).coords[0]
    geom = transformer.transform(coords[0], coords[1])
    row['Lat'] = geom[0]
    row['Lng'] = geom[1]
    return row

pv_merged = pv_merged.apply(find_latlng, axis=1)

# list(lion_df.columns)
# will join dataset of parking violations columns Registration State, 

In [127]:
pv_merged.to_csv('latlng.csv')